In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import brute
from tqdm import tqdm

plt.style.use("seaborn")

In [2]:
df = pd.read_csv("/Users/maxibertonalbornoz/Documents/Python/Udemy/Algorithmic Trading A-Z with Python and ML/Part3_Materials/intraday.csv", parse_dates=["time"], index_col="time")
df

,price
time,
2018-01-01 22:00:00+00:00,1.201205
2018-01-02 04:00:00+00:00,1.207055
2018-01-02 10:00:00+00:00,1.204440
2018-01-02 16:00:00+00:00,1.205800
2018-01-02 22:00:00+00:00,1.204690
...,...
2019-12-29 22:00:00+00:00,1.119920
2019-12-30 04:00:00+00:00,1.119940
2019-12-30 10:00:00+00:00,1.120095


In [27]:
def run_strategy(boll):
    data = df.copy()
    data["returns"] = np.log(data.price.div(data.price.shift(periods=1)))
    
    data["SMA"] = data["price"].rolling(window=boll[0]).mean()
    data["Upper"] = data.SMA + data.price.rolling(window=boll[0]).std().mul(boll[1])
    data["Lower"] = data.SMA - data.price.rolling(window=boll[0]).std().mul(boll[1])
    data.dropna(inplace=True)

    data["distance"] = (data.price - data.SMA)

    data["position"] = np.where(data.price < data.Lower, 1, np.nan)
    data["position"] = np.where(data.price > data.Upper, -1, data["position"])
    data["position"] = np.where(data.distance * data.distance.shift(periods=1)<0, 0, data["position"])
    data["position"] = data.position.ffill().fillna(0)

    data["strategy"] = data.position.shift(periods=1).mul(data.returns)
    data.dropna(inplace=True)

    return -data[["returns", "strategy"]].sum().apply(np.exp)[1]

In [28]:
run_strategy((30,2))

-1.0781362437114343

In [29]:
brute(run_strategy, ((20,100,1), (1,5,1)), finish=None)

array([58.,  1.])